In [ ]:
!nvidia-smi

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))


NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

Your runtime has 38.0 gigabytes of available RAM



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
data = np.load('BIDMCPPGREP125.npy')

In [ ]:
import pandas as pd
BIDMCPPGREP_df = pd.DataFrame(data=data,dtype="float64")
BIDMCPPGREP_df = BIDMCPPGREP_df.dropna()
BIDMCPPGREP_df

/usr/local/lib/python3.6/dist-packages/pandas/core/internals/construction.py:195: ComplexWarning: Casting complex values to real discards the imaginary part
  values = values.astype(dtype)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,-0.011756,-0.044888,0.050812,-0.053975,0.022543,-0.005991,-0.048628,0.096319,-0.080637,-0.153027,0.012271,0.218598,-0.640089,0.886683,94.000000,92.0,24.999970,97.0,2.0
1,-0.026074,0.026924,-0.001888,-0.034464,0.102838,-0.075853,0.036893,-0.008456,-0.079782,-0.000132,-0.022780,0.055642,-0.831237,0.800837,94.000000,92.0,24.999970,97.0,2.0
2,-0.027053,0.100141,-0.068141,0.056931,-0.025746,-0.012675,0.084528,-0.086499,0.119788,-0.224944,0.001221,0.069195,-1.313228,0.527022,94.000000,92.0,24.999970,97.0,2.0
3,-0.008486,-0.005976,-0.013808,0.002903,-0.031970,0.038968,-0.019237,0.001659,-0.040172,0.046067,-0.249103,0.042718,-0.360133,0.802967,92.000000,92.0,26.000031,97.0,2.0
4,-0.060819,0.010487,-0.001208,-0.009710,-0.007809,-0.028824,-0.010964,-0.037729,0.238712,-0.069395,0.059211,-0.299983,-1.201739,0.417319,93.000000,92.0,26.000031,97.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25435,0.105646,-0.038316,-0.035939,0.174715,-0.000741,-0.178416,0.131630,-0.382381,0.068487,-0.579359,0.099840,-0.399977,-0.268802,0.488797,91.000092,94.0,17.999924,100.0,2.0
25436,0.060428,-0.011845,0.147132,0.052500,0.069557,-0.026635,-0.015387,0.014457,-0.030509,-0.490517,0.058154,-0.239012,1.943018,1.465372,91.999954,95.0,17.999924,100.0,2.0
25437,-0.011693,0.108399,-0.053722,0.104793,0.292719,-0.018628,0.015258,0.037115,-0.192577,-0.027394,-0.172174,-0.512782,1.419629,1.243960,89.999847,95.0,18.999909,100.0,2.0
25438,0.122031,-0.021316,-0.309890,0.244475,-0.161251,0.119614,-0.058787,-0.130610,-0.078804,0.314449,0.037731,-0.048909,1.358749,1.354423,97.000031,95.0,18.999909,100.0,2.0


In [ ]:
X = BIDMCPPGREP_df.iloc[:,:-1]
Y = BIDMCPPGREP_df.iloc[:,-1]
data_x =  X.to_numpy(dtype="float32")
data_y = Y.to_numpy(dtype="float32")
data_x = data_x.reshape(len(data_x),len(data_x[0]),1)

In [ ]:
644,324,'batch' ,16,32

(644, 324, 'batch', 16, 32)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Bidirectional,LayerNormalization
from keras.layers import GRU
from keras.metrics import MeanSquaredError,MeanAbsoluteError,TruePositives,TrueNegatives,FalsePositives,FalseNegatives,PrecisionAtRecall,SensitivityAtSpecificity,Recall,Precision
from keras.utils import np_utils
from keras.models import load_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.preprocessing import LabelEncoder

X=data_x
Y=data_y
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
dummy_y = np_utils.to_categorical(encoded_Y)

test=[0.10,0.25,0.50]
#test=[0.50]
mnames = ['32_4_10_64','32_4_25_64','32_4_50_64']
#mnames = ['64_4_50_16']
for i in range(0,len(test)):
    X_train, X_test, Y_train, Y_test = train_test_split(X, dummy_y , test_size=test[i])

    model = Sequential()
    model.add(GRU(64,return_sequences=True))
    model.add(LayerNormalization())
    model.add(Dropout(0.3))
    model.add(GRU(32,return_sequences=True))
    model.add(LayerNormalization())
    model.add(Dropout(0.3))
    model.add(GRU(4))
    model.add(Dense(64,activation='relu'))
    model.add(LayerNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(32,activation='relu'))
    model.add(LayerNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(4,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',
                  metrics=['accuracy','MeanSquaredError','MeanAbsoluteError','TruePositives',
                           'TrueNegatives','FalsePositives','FalseNegatives','Recall','Precision'])
    history = model.fit(X_train,Y_train ,batch_size=16,epochs=50,validation_data=(X_test, Y_test))
    model.save('PPGFModel'+mnames[i]+'.h5')
    df = pd.DataFrame(history.history)
    df.to_csv('PPGFModel'+mnames[i]+'.csv')

Epoch 1/50
1136/1431 [======================>.......] - ETA: 4s - loss: 1.3415 - accuracy: 0.3453 - mean_squared_error: 0.1817 - mean_absolute_error: 0.3540 - true_positives: 1431.0000 - true_negatives: 53443.0000 - false_positives: 1085.0000 - false_negatives: 16745.0000 - recall: 0.0787 - precision: 0.5688